VT5

In [69]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('default')

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [70]:
laliga_df2016_2017 = pd.read_excel("..\\data\\laligadata\\LaLiga16 17.xlsx").dropna(how='all')
laliga_df2017_2018 = pd.read_excel("..\\data\\laligadata\\LaLiga17 18.xlsx").dropna(how='all')
laliga_df2018_2019 = pd.read_excel("..\\data\\laligadata\\LaLiga18 19.xlsx").dropna(how='all')
laliga_df2019_2020 = pd.read_excel("..\\data\\laligadata\\LaLiga19 20.xlsx").dropna(how='all')
laliga_df2020_2021 = pd.read_excel("..\\data\\laligadata\\LaLiga20 21.xlsx").dropna(how='all')
laliga_df2021_2022 = pd.read_excel("..\\data\\laligadata\\LaLiga21 22.xlsx").dropna(how='all')

In [71]:
laliga = pd.concat([laliga_df2016_2017, laliga_df2017_2018, laliga_df2018_2019, laliga_df2019_2020, laliga_df2020_2021, laliga_df2021_2022], ignore_index=True)

In [72]:
laliga = laliga.loc[:,["Wk",'Date',"Local","Score",'Visitante','Attendance']]

In [73]:
laliga['Attendance'] = laliga['Attendance']*1000
laliga['Torneo'] = 'Primera division'

In [74]:
laliga.head(2)

,Wk,Date,Local,Score,Visitante,Attendance,Torneo
0,1.0,2016-08-19,Málaga,1–1,Osasuna,22347.0,Primera division
1,1.0,2016-08-19,La Coruña,2–1,Eibar,21441.0,Primera division


In [75]:
def golesvisita(marcador):
    goles=marcador.split("–")
    return int(goles[1])
def goleslocal(marcador):
    goles=marcador.split("–")
    return int(goles[0])

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Total goles partido'] = laliga['Goles Local'] + laliga['Goles Visitante']

In [76]:
def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

    
laliga['Año'] = laliga['Date'].dt.year
laliga['Mes'] = laliga['Date'].dt.month
laliga['Temporada'] = laliga.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)

In [77]:
def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'
    
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
laliga['Resultado'] = laliga['Score'].apply(resultado)
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)

In [78]:
laliga.head(5)

,Wk,Date,Local,Score,Visitante,Attendance,Torneo,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,1.0,2016-08-19,Málaga,1–1,Osasuna,22347.0,Primera division,1,1,2,2016,8,16/17,Empate,Empate
1,1.0,2016-08-19,La Coruña,2–1,Eibar,21441.0,Primera division,2,1,3,2016,8,16/17,Local,La Coruña
2,1.0,2016-08-20,Barcelona,6–2,Betis,65731.0,Primera division,6,2,8,2016,8,16/17,Local,Barcelona
3,1.0,2016-08-20,Granada,1–1,Villarreal,15149.0,Primera division,1,1,2,2016,8,16/17,Empate,Empate
4,1.0,2016-08-20,Sevilla,6–4,Espanyol,29420.0,Primera division,6,4,10,2016,8,16/17,Local,Sevilla


Evolucion

In [79]:
ej2122 = laliga[(laliga['Temporada']=='21/22')&(laliga['Torneo']=='Primera division')]
equip2122 = ej2122['Local'].unique().tolist()

In [80]:
def tabla_posicion(data,fecha,equipos):
    tabla = {}
    for equipo in equipos:
        data_usar = data[(data['Wk']<=fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
        puntos = 0
        for i in range(len(data_usar)):
            ganador = str(data_usar.iloc[i]['Ganador'])
            if ganador == equipo:
                puntos += 3
            if ganador == 'Empate':
                puntos +=1
        tabla[equipo] = puntos
    df = pd.DataFrame([[key, tabla[key]] for key in tabla.keys()], columns=['Equipo', 'Puntos'])
    df = df.sort_values(by='Puntos',ascending = False).reset_index()
    return df[['Equipo','Puntos']]

In [81]:
tabla_33 = tabla_posicion(ej2122,33,equip2122)
tabla_33

,Equipo,Puntos
0,Real Madrid,78
1,Barcelona,63
2,Sevilla,63
3,Atlético Madrid,61
4,Betis,57
5,Real Sociedad,55
6,Villarreal,52
7,Athletic Club,48
8,Osasuna,44
9,Valencia,42


In [82]:
ej2122 = laliga[(laliga['Temporada']=='21/22')&(laliga['Torneo']=='Primera division')]
equip2122 = ej2122['Local'].unique().tolist()

In [94]:
def tabla_posicion_equipo(data,fecha,equipos,eq):
    tabla = {}
    for equipo in equipos:
        data_usar = data[(data['Wk']<=fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
        puntos = 0
        for i in range(len(data_usar)):
            ganador = str(data_usar.iloc[i]['Ganador'])
            if ganador == equipo:
                puntos += 3
            if ganador == 'Empate':
                puntos +=1
        tabla[equipo] = puntos
    df = pd.DataFrame([[key, tabla[key]] for key in tabla.keys()], columns=['Equipo', 'Puntos'])
    df = df.sort_values(by='Puntos',ascending = False).reset_index()
    df = df[['Equipo','Puntos']]
    pos = df[df['Equipo']==eq].index.values[0]
    return pos+1

In [95]:
fechas = sorted(ej2122['Wk'].unique().tolist())

In [96]:
fecha = 38
equipo = "Betis"

In [97]:
posiciones = []
for fec in fechas:
    pos = tabla_posicion_equipo(ej2122,fec,equip2122,equipo)
    posiciones.append(pos)
    if fec == fecha:
        break

In [98]:
posiciones

[15,
 15,
 17,
 11,
 13,
 10,
 7,
 11,
 8,
 5,
 4,
 5,
 5,
 5,
 5,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [ ]:
cond_predominante_abs = laliga.groupby(['Resultado'])['Wk'].count().reset_index()
cond_predominante_abs = cond_predominante_abs.set_axis(['Resultado', 'Cantidad'], axis='columns')

In [ ]:
cond_predominante_abs

In [ ]:
fig1 = px.pie(cond_predominante_abs, 
              values='Cantidad', 
              names='Resultado', 
              title='La liga 2016-2022',
              color='Resultado',
              color_discrete_map={'Local':'royalblue','Visitante':'orangered','Empate':'lightgreen'})

fig1.update_traces(textposition='inside', textinfo='percent+value')
fig1.show()

In [ ]:
total_asist=laliga.groupby('Temporada')['Attendance'].agg('sum').reset_index()
fig2 = px.line(total_asist, 
               x='Temporada', 
               y='Attendance',
               title = 'Asistencia total por temporada de la Liga entre 2016-2022', line_shape='spline')
fig2.show()

In [ ]:
test1 = laliga.groupby('Local')['Goles Local'].agg('sum').reset_index()
test1 = test1.rename(columns={'Local':'Equipo', 'Goles Local': 'Total de goles históricos de Local'})

test2 = laliga.groupby('Visitante')['Goles Visitante'].agg('sum').reset_index()
test2 = test2.rename(columns={'Goles Visitante': 'Total de goles históricos de Visitante'})

In [ ]:
result = pd.concat([test1, test2], axis=1)
dt = result.drop(['Visitante'], axis=1)

In [ ]:
fig3 = px.bar(dt,
              x="Equipo",
              y=['Total de goles históricos de Local', 'Total de goles históricos de Visitante'],
              title='Total de goles desde 2016-2022 Equipos de LaLiga')
fig3.show()

# Construir un dashboard

In [ ]:
laliga.columns

In [ ]:
laliga = laliga.loc[:,['Wk','Date', 'Local', 'Score', 'Visitante', 'Temporada','Attendance']]
segunda = segunda.loc[:,['Wk','Date', 'Local', 'Score', 'Visitante', 'Temporada','Attendance']]

In [ ]:

segunda['Attendance'] = segunda['Attendance']*1000
segunda['Torneo'] = 'Segunda division'

In [ ]:
partidos6t = pd.concat([laliga,segunda], axis=0)

In [ ]:
partidos6t

In [ ]:
agrup_tortemp = partidos6t.groupby(['Torneo', 'Temporada'])['Attendance'].sum().reset_index()

In [ ]:
agrup_tortemp

In [ ]:
asist_pub = pd.DataFrame({
    "Primera division": agrup_tortemp[agrup_tortemp['Torneo']=='Primera division']['Attendance'].to_list(),
    "Segunda division": agrup_tortemp[agrup_tortemp['Torneo']=='Segunda division']['Attendance'].to_list()},
    index=agrup_tortemp[agrup_tortemp['Torneo']=='Primera division']['Temporada'].to_list()
    )

In [ ]:
asist_pub

In [ ]:
p_line = asist_pub.plot_bokeh(kind="line", xlabel='Temporada', ylabel='Asistencia total', show_figure=False)